# CI/CD for a Kubeflow pipeline on Vertex AI

**Learning Objectives:**
1. Learn how to create a custom Cloud Build builder to pilote Vertex AI Pipelines
1. Learn how to write a Cloud Build config file to build and push all the artifacts for a KFP
1. Learn how to setup a Cloud Build GitHub trigger a new run of the Kubeflow PIpeline

In this lab you will walk through authoring of a **Cloud Build** CI/CD workflow that automatically builds, deploys, and runs a Kubeflow pipeline on Vertex AI. You will also integrate your workflow with **GitHub** by setting up a trigger that starts the  workflow when a new tag is applied to the **GitHub** repo hosting the pipeline's code.

## Configuring environment settings

In [1]:
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
REGION = "us-central1"
ARTIFACT_STORE = f"gs://{PROJECT_ID}-kfp-artifact-store"

Let us make sure that the artifact store exists:

In [2]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-03-2de90311f4ec-kfp-artifact-store/


## Creating the KFP CLI builder for Vertex AI
### Review the Dockerfile describing the KFP CLI builder

In [3]:
!cat kfp-cli_vertex/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install kfp==1.6.6
RUN pip install google-cloud-aiplatform==1.3.0
ENTRYPOINT ["/bin/bash"]


### Build the image and push it to your project's **Container Registry**.

In [4]:
KFP_CLI_IMAGE_NAME = "kfp-cli-vertex"
KFP_CLI_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{KFP_CLI_IMAGE_NAME}:latest"
KFP_CLI_IMAGE_URI

'gcr.io/qwiklabs-asl-03-2de90311f4ec/kfp-cli-vertex:latest'

In [5]:
!gcloud builds submit --timeout 15m --tag {KFP_CLI_IMAGE_URI} kfp-cli_vertex

Creating temporary tarball archive of 2 file(s) totalling 1.1 KiB before compression.
Uploading tarball of [kfp-cli_vertex] to [gs://qwiklabs-asl-03-2de90311f4ec_cloudbuild/source/1690441573.260655-350f6c57c6604023879f89a653dd3325.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-03-2de90311f4ec/locations/global/builds/ef421acd-f213-4757-944e-398d4a7dbb77].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/ef421acd-f213-4757-944e-398d4a7dbb77?project=94255171416 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "ef421acd-f213-4757-944e-398d4a7dbb77"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-03-2de90311f4ec_cloudbuild/source/1690441573.260655-350f6c57c6604023879f89a653dd3325.tgz#1690441573496308
Copying gs://qwiklabs-asl-03-2de90311f4ec_cloudbuild/source/1690441573.260655-350f6c57c6604023879f89a653dd3325.tgz#1690441573496308...
/ [1 files][  900.0 B/  900.0 B]                 

## Understanding the **Cloud Build** workflow.

Review the `cloudbuild_vertex.yaml` file to understand how the CI/CD workflow is implemented and how environment specific settings are abstracted using **Cloud Build** variables.

The CI/CD workflow automates the steps you walked through manually during `lab-02_vertex`:
1. Builds the trainer image
1. Compiles the pipeline
1. Uploads and run the pipeline to the Vertex AI Pipeline environment
1. Pushes the trainer to your project's **Container Registry**
 

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **KFP CLI**. 

## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the [gcloud builds submit command]( https://cloud.google.com/sdk/gcloud/reference/builds/submit).

In [6]:
SUBSTITUTIONS = f"_REGION={REGION},_PIPELINE_FOLDER=./"
SUBSTITUTIONS

'_REGION=us-central1,_PIPELINE_FOLDER=./'

In [7]:
!gcloud builds submit . --config cloudbuild_vertex.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 12 file(s) totalling 38.9 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-asl-03-2de90311f4ec_cloudbuild/source/1690441773.947376-b1ad745e06b945809a20562a6eababdf.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-03-2de90311f4ec/locations/global/builds/78abd048-f704-46fb-aca8-19767a0c00d2].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/78abd048-f704-46fb-aca8-19767a0c00d2?project=94255171416 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "78abd048-f704-46fb-aca8-19767a0c00d2"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-03-2de90311f4ec_cloudbuild/source/1690441773.947376-b1ad745e06b945809a20562a6eababdf.tgz#1690441774177168
Copying gs://qwiklabs-asl-03-2de90311f4ec_cloudbuild/source/1690441773.947376-b1ad745e06b945809a20562a6eababdf.tgz#1690441774177168...
/ [1 files][  8.3 KiB/  8.3 KiB]                            

**Note:** If you experience issues with CloudBuild being able to access Vertex AI, you may need to run the following commands in **CloudShell**:

```
PROJECT_ID=$(gcloud config get-value project)
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)")

gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member serviceAccount:$PROJECT_NUMBER@cloudbuild.gserviceaccount.com \
  --role roles/editor
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member serviceAccount:$PROJECT_NUMBER-compute@developer.gserviceaccount.com \
    --role roles/storage.objectAdmin
```

## Setting up GitHub integration

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

### Create a fork of this repo
[Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork [this repo](https://github.com/GoogleCloudPlatform/asl-ml-immersion)

### Create a **Cloud Build** trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location| ./notebooks/kubeflow_pipelines/cicd/solutions/cloudbuild_vertex.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_REGION|us-central1|
|_PIPELINE_FOLDER|notebooks/kubeflow_pipelines/cicd/solutions

### Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


After running the command above, a build should have been automatically triggered, which you should able to inspect [here](https://console.cloud.google.com/cloud-build/builds).

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.